In [1]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device ='cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size=8
batch_size=4
max_iters = 1000
learning_rate = 3e-4
eval_iters =250


cuda


In [2]:
with open('book.txt','r', encoding='utf-8') as f:
    text= f.read()
chars=sorted(set(text))
print(chars)
vocab_size=(len(chars))

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'Â', 'Æ', 'È', 'É', 'Ü', 'à', 'ç', 'è', 'é', 'ê', 'ë', 'í', 'ï', 'ô', 'ü', '\ufeff']


In [3]:
string_to_int = {ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data= torch.tensor(encode(text), dtype=torch.long)

print(data[:100])

tensor([95,  0, 44, 32, 29,  1, 37, 33, 38, 28,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 26, 42, 25, 33, 38,  0,  0, 55, 78,  0,  0, 25, 36, 30, 42, 29, 28,
         1, 26, 33, 38, 29, 44,  0,  0, 28, 62, 71, 58, 56, 73, 58, 74, 71,  1,
        57, 74,  1, 36, 54, 55, 68, 71, 54, 73, 68, 62, 71, 58,  1, 57, 58,  1,
        40, 72, 78, 56, 61, 68, 65, 68, 60, 62, 58,  0, 85,  1, 65, 54,  1, 43,
        68, 71, 55, 68, 67, 67, 58,  0,  0, 26])


In [4]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


def get_batch(split):
    data =train_data if split =='train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    #print(ix)
    x=torch.stack([data[i:i+block_size] for i in ix])
    y= torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x,y = get_batch('train')
print('inputs:')
print(x)
print('targets:')
print(y)

inputs:
tensor([[67, 58, 72, 72,  1, 57, 62, 58],
        [72, 56, 65, 58,  1, 69, 71, 68],
        [62, 72,  1, 73, 71, 74, 58,  1],
        [72,  1, 67, 68, 73,  1, 62, 67]], device='cuda:0')
targets:
tensor([[58, 72, 72,  1, 57, 62, 58, 24],
        [56, 65, 58,  1, 69, 71, 68, 57],
        [72,  1, 73, 71, 74, 58,  1, 73],
        [ 1, 67, 68, 73,  1, 62, 67, 56]], device='cuda:0')


In [5]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
        model.train()
    return out

In [6]:
from torch import nn
import torch.nn as nn

class BigramLanguageModel(nn.Module):
    def __init__(self,vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, index,targets = None):
        logits = self.token_embedding_table(index)
       
        if targets is None:
           loss = None
        else:
          B, T, C = logits.shape
          logits = logits.view(B*T, C)
          targets = targets.view(B*T)
          loss = F.cross_entropy(logits, targets)

        return logits, loss

    
    def generate(self, index, max_new_tokens):
       #index is *B,T) array of indices in the current context
       for _ in range(max_new_tokens):
          #get the predictions
          logits, loss = self.forward(index)
          #focus onlly on the last time step
          logits = logits[:, -1, :] # becomes (B,C)
          #apply softmax to get probablilites
          probs = F.softmax(logits, dim =-1)
          #sample from distribution
          index_next = torch.multinomial(probs, num_samples=1)
          index = torch.cat((index, index_next), dim=1)
        
       return index
    
model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1),dtype = torch.long, device = device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


us![ígvFQ3èU
EmNq0Q?GïëZ*fi,*fP8Y1vA[Fgü-.XfÂyazêCfBcÜ:6ÉÆcüdaMvüïIëoyzUqs![8íé]8gD
26cjÈé
,L.qdcÉn?íy&4ÂlRFh6fÂX]íLMh,sl.IVp*5J;1jyczdCfçDFDLkVo5g[È﻿7EÉ;rtT9NbÈ(éFUíZ8Q&,Un_Æb75CHxhïLyüN"mW7:0Mà;rCüèüïLD];ëYàÜVQ';w7uG2aq 0D*4"èSg0vOÆH]féL0Rêruf3éecm2Në4"gOïLrCJjNcj"tC
CfCoÉ'!ü
CH6VwzAaÜíqdNÈ(),7m4VtCHY1fëM;18jkíQë-r[çt_HJrCHYrF:alyM gï,JcUn﻿fekVa)gowsTOçNÂ3OíQ[4ÂM3EZ!N(ÉP0npqn ÆiFÆê5pp*hÜAT727jqLD6p u(v!oGQüÈ(ébô5l.I0X4!ëí
_d!ÆgêC86s9dBm]7;eChhÈurü

&pAÈ(QKh8Æi hf"y&B5Cê4﻿_üPçy:HBkÉ4ZGns!:pME![


In [7]:


optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iter in range(max_iters):
    
    if iter % eval_iters ==0:   ##checks if remainder of current iteration by eval iter is 0 
        losses= estimate_loss()
        
        print(f"step: {iter}, train loss: {losses['train']:.3f}, val loss: {losses['val']:.3f}")   
    #sample a batch of data
    xb, yb = get_batch('train')

    ##evaluate the loss
    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0, train loss: 5.057, val loss: 5.041
step: 250, train loss: 4.981, val loss: 4.981
step: 500, train loss: 4.934, val loss: 4.921
step: 750, train loss: 4.851, val loss: 4.849
4.826185703277588


In [8]:
context = torch.zeros((1,1), dtype =torch.long, device= device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


ôïc-h.Vx﻿ZVus!o﻿_Â;7Mh
ôpK6dYS8È2çPy[këJ çq9Uühc]&﻿-6Én"GXCcVôQÂ
Y]d1bZàw)Xhh90(Otty1x﻿8hgQ9Rs!PÆDA*HYuCmG)NÈ(jFÜ7çy&_ E_eCH.x,m7 YhcÜS&!E4hdhígjpX7Aj&É
PQ2&bSèK9[gc4S6ÜhïQUe1BC)ë4_çkQ?KGpM:usë9ôGB*4: y:uÆf.8sÈï!EÉRLÈ("I8X1'?tÂ F*c([KOb0R3êrônCHOlMHl﻿aJoÉI1UJOüÈ(S3È(e'HuiQÉíIQ2GQve)Éüb52uJ -KÜíTa!:ï?oíïWs]&È'àY!degEjEtëLg-sQt4Elf tyS)ph_Jé0R!Jm7YyÉPScjN78hïL.C]zq!5ELXfZQaYÂ&﻿ÈyMb HvX(WjDk6éT﻿2g-éÈX, KbTv-ü
ZÂ?]Vu.9u0y8ÉjZuFDíhÈ3Wvsí7mnVQK6?osA1*F7YpérB&,5RPqí
vqs,èU bl'pPf)çq4XB&É.nJJé7fëGF﻿N'Hu
